In [117]:
import numpy as np
import timeit
import pandas as pd


## Definições
<hr/>

In [118]:
def f(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return 3**(1/(x1+x2))+x1**2+x2**2

In [119]:
def Gradiente(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return np.array([[2*x1-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2],
            [2*x2-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2]])

In [120]:
def Heissiana(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return np.array([[2 - (np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2))))/(x1+x2)**4, -np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**4],

            [-np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**4 ,2 - (np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2))*(x1+x2)))/(x1+x2)**4]])

In [121]:
def definida_positiva_2x2(matrix):    
    if matrix[0][0] > 0 and np.linalg.det(matrix)>0:
        return True
    else: 
        return False

In [122]:
def is_invertible(a):
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]

In [123]:
def test_combinations(search, method, maxiter, tol):
    result = {
        "start": [],
        "gamma": [],
        "eta": [],
        "Armijo Iterations": [],
        "Method Iterations": [],
        "Optical Point Found": [],
        "Optical Value Found": [],
        "Time": []
    }
    for point in search['start']:
        for gamma in search['gamma']:
            for eta in search['eta']:
                start = timeit.default_timer()
                met =  method(point, tol, maxiter, gamma, eta)
                stop = timeit.default_timer()
                result["start"].append(point)
                result["gamma"].append(gamma)
                result["eta"].append(eta)
                result["Armijo Iterations"].append(met[2])
                result["Method Iterations"].append(met[1])
                result["Optical Point Found"].append(met[0])
                result["Optical Value Found"].append(f(met[0]))
                result["Time"].append(stop - start)

    return result

In [124]:
search = {
    "start": [
        [[0.5], [0.5]], 
        [[-1], [-1]], 
        [[-6000], [-5000]],
        [[6000],[5000]],
        [[5], [-0.5]],
        [[-5], [0.5]] 
    ],
    "gamma": [
        0.8,
        0.65,
        0.5
    ],
    "eta": [
        0.25,
        0.1
    ],
}

# Estudo da Função
<hr/>

## Primeira foto da função

![Geogebra function](images/function.png)



Apenas pela observação direta da função podemos visualizar criada pelo geogebra:
1. Há uma uma descontinuidade na função na reta (x = -y);
2. Há um mínimo local no primeiro quadrante (x > 0 e y > 0);
3. No domínio observado, a função é convexa.


## Decontinuidade

Devido a razão $\frac{1}{x_1 + x_2}$ presente no exponente do 3, temos que:

$$
\lim_{x_1 \to -x_2} f(x_1, x_2) = \infty \text{  e  } \lim_{x_2 \to -x_1} f(x_1, x_2) = \infty
$$
Consequentemente,
$$
\lim_{(x_1, x_2) \to (0, 0)} f(x_1, x_2) = \infty
$$
Isso significa que a função não está definida no ponto (0, 0) e, portanto, ele não pode ser o mínimo global da função como aparenta na imagem.\\


## Pontos estacionários

Para calcular os pontos estácionários da função, vamos igualar seu gradiante a zero

$$
\nabla f(x_1, x_2) = \begin{bmatrix}
        2x - \frac{(\ln{3}e^{\frac{\ln{3}}{x + y}})}{(x + y)^2}\\
        2y - \frac{(\ln{3}e^{\frac{\ln{3}}{x + y}})}{(x + y)^2}
\end{bmatrix} = 0
$$

Assim, obtemos um sistema de equações que pode ser resolvido subtraindo a primeira equação da segunda:

$$
        2x - 2y =  0\\
        x = y
$$

Isso indica que os pontos estacionários estão no domnínio estabelecido pela reta x = y. <br>
Vamos calcular e visualizar o que acontece quando o hiperplano formado pela reta intercepta a função.

$$
        g(x) = f(x, x) = 3^{\frac{1}{2x}} + 2x^2
$$


Intersseção em 3D            |  Intersseção em 2D (curva formada no eixo z e no eixo x)
:-------------------------:|:-------------------------:
![](images/hiperplano.png)  |  ![](images/intersse.png)



Conseguimos observar o <strong>mínimo local</strong> mencionado anteriormente e aproximação feita pelo Geogebra do seu valor, mas gostaríamos de calcular esse valor numericamente. <br>Como não foi possível terminar a resolução do sistema de equações acima com os métodos tradicionais, optamos por usar uma <strong>aproximação de Taylor de terceira ordem </strong> para aproximar a função $3^{\frac{1}{2x}}$ na forma de um polinômio.


 - Série de Taylor desenvoldida:
$$
        3^{\frac{1}{2x}} \approx \sum_{n = -3}^0 \frac{x^n (\frac{2}{\ln{3}})^n}{(-n!)} = 1\:+\:\frac{0.5493061443340549}{x}\:+\:\frac{0.15086862010157276}{x^2}+\frac{0.02762435333633141}{x^3}
$$
<p align="center">
  <img src="images/taylo.png" />
</p>




In [125]:
def CoeffTaylor(n):
    return (2/np.log(3))**n/(np.math.factorial(-n))
CoeffTaylor(0), CoeffTaylor(-1), CoeffTaylor(-2), CoeffTaylor(-3)

(1.0, 0.5493061443340549, 0.15086862010157276, 0.02762435333633141)


Dessa forma, foi possível encontrar um valor aproximadO para o ponto crítico da função algebricamente:

$$
        
        g'(x) = 4x-\frac{0.5493}{x^2}-\frac{0.30172}{x^3}-\frac{0.08286}{x^4}\\
         4x-\frac{0.5493}{x^2}-\frac{0.30172}{x^3}-\frac{0.08286}{x^4} = 0\\
        \therefore x = 0.66714
$$
O ponto estacionário encontrado foi ~(0.66714, 0.66714) com f(x, y) = 16832, que é muito parecido com o ponto aproximado pelo software ~(0.67633,0.67633) com f(x, y) = 3.16767


## Convexidade

Para analisar a convexidade vamos avaliar a Hessiana da função:

Seja w = $\frac{\ln{(3)} (\ln{(3)} e^{\frac{\ln{3}}{x+y}} + 2 (x + y) e^{\frac{\ln{3}}{x+y}})}{(x+y)^4}$, a Hessiana pode ser escrita da seguinte forma:


$$
H = \begin{bmatrix}
2+w & w \\
w & 2+w 
\end{bmatrix}
$$
Sabemos que para um matriz ser definida positiva, a sua determinante e a determinante de suas submatrices precisa ser maior que zero:


$$
    det([2+w]) = 2 + w > 0 \\
    det(H) = (2+w)^2 - w^2 > 0
$$

Se x > 0 e y > 0, w é sempre postivo o que torna as duas desigualdades acima verdadeiras. Então, a matriz é definida positiva nesse domínio e portanto convexa. Isso já indica que nosso ponto estacionário é um ponto de mínimo, vamos conferir:




In [126]:
def definida_positiva_2x2(matrix):    
    if matrix[0][0] > 0 and np.linalg.det(matrix)>0:
        return True
    else: 
        return False
    
heissiana_ponto_critico = Heissiana([[0.66714],[0.66714]])
definida_positiva_2x2(heissiana_ponto_critico)

True

Provamos, então, que o ponto crítico encontrado nesta análise é um ponto de <strong>mínimo</strong>. <br>
É fácil perceber que ele é <strong>local</strong>, pois existem valores em que a função é menor.


In [127]:
f([[-1], [-1]])

2.5773502691896257

## Apenas mínimo local 

A função analisada é um caso exepcional em que há um mínimo local, mas não há um mínimo global. <br>

Pela definição: c é um mínimo global da função f(x), se f(c)≥f(x) para todo x no domínio de f.<br>
Como o ponto (0, 0) não está definido e ao domínio da função é $\mathcal{R^2}$, sempre podemos encontrar um ponto c cada vez mais próximo de zero que torna a função cada vez menor, o que foge da definição.




# Estudo dos Pontos 
<hr/>

<p align="center">
  <img src="images/Pontos.Jpeg" />
</p>

- Dividimos nossa função em duas partes: 
    1) Contém o ponto descontínuo (0,0).
    2) Contém o mínimo local.

- Separamos 6 pontos tendo como base na parte da função e quadrante do domínio.
    - O ponto (-1,-1) (ponto A da imagem) que está próximo ao ponto não definido de f(x). Esperamos que este ponto  convirja para um ponto próximo de (0,0), até chegar na tolerância devido a sua proximidade a descontinuidade.
    - P ponto (0.5,0.5) (Ponto B da imagem), ponto próximo ao mímimo local de f(x). Esperamos que este ponto convirja para o mínimo local.
    - Os pontos (-6000,-5000) e (6000,5000) que são suficientemente grandes para vermos o funcionamento dos métodos com números grandes e a convergencia do método. O fato desses valores serem grandes pode fazer com que o deslocamento seja grande mesmo com um passo pequeno e queríamos estudar esse comportamento.
    - O ponto (5, -0.5), pertencente a segunda parte (ponto C), e (-5, 0.5), pertencente a primeira parte (ponto D), são pontos ligeiramente distantes do ponto de descontinuidade e do ponto mínimo de f(x) respectivamente. Esperamos que cada ponto tenha convergencia a pontos de suas respectivas partes dependendo do tamanho do passo. Isso porque é necessário um passo suficientemente grande para o ponto D fugir da descontinuidade e convergir para o mínimo logal e, por outro lado, passos grandes podem fazer com que C caia na descontinuidade e não convirja.

# Implementação dos métodos
<hr>

### Busca de Armijo

In [128]:
def Armijo(x,gama,d,n):
    t = 1
    k = 0
    d = np.array(d)
    x = np.array(x)
    while f(x + t*d) > f(x) + n*t*(np.transpose(Gradiente(x))@d):
        t=gama*t
        k = k+1
    return [t,k]

### Método do Gradiente

In [129]:
def met_gradiente(x,tol,maxiter,gama,n, debug=False):
    x = np.array(x)
    m = 0
    k = 0
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        d = - Gradiente(x)
        ar = Armijo(x,gama,d,n)
        k = k + ar[1]
        t = ar[0]
        x = x + t*d
        m = m + 1
    return [x,m,k] 

In [130]:
tol = 0.0001
maxiter = 1000
result = test_combinations(search, met_gradiente, maxiter, tol)       
grad_res = pd.DataFrame.from_records(result)    
grad_res[grad_res["eta"] == 0.25]

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


,Armijo Iterations,Method Iterations,Optical Point Found,Optical Value Found,Time,eta,gamma,start
0,73,9,"[[0.6763370143192518], [0.6763370143192518]]",3.167673e+00,0.003673,0.25,0.80,"[[0.5], [0.5]]"
2,30,7,"[[0.6763255367588685], [0.6763255367588685]]",3.167673e+00,0.001174,0.25,0.65,"[[0.5], [0.5]]"
4,13,4,"[[0.6763349644096142], [0.6763349644096142]]",3.167673e+00,0.000591,0.25,0.50,"[[0.5], [0.5]]"
6,26,6,"[[-8.873752021730419e-06], [-8.873752021730419...",1.574869e-10,0.001411,0.25,0.80,"[[-1], [-1]]"
8,13,6,"[[-1.1955435295191822e-05], [-1.19554352951918...",2.858649e-10,0.000850,0.25,0.65,"[[-1], [-1]]"
10,22,11,"[[-1.8035048834030365e-05], [-1.80350488340303...",6.505260e-10,0.001830,0.25,0.50,"[[-1], [-1]]"
12,127,23,"[[0.6763537339011317], [0.676328007051467]]",3.167673e+00,0.010243,0.25,0.80,"[[-6000], [-5000]]"
14,20,13,"[[-3.621000219235728e-05], [-1.185998004600115...",1.451823e-09,0.003386,0.25,0.65,"[[-6000], [-5000]]"
16,48,24,"[[-3.534388699473392e-05], [2.4260757780656653...",1.837775e-09,0.004341,0.25,0.50,"[[-6000], [-5000]]"
18,34,12,"[[-4.091205876717821e-05], [-1.484462071771027...",1.894159e-09,0.002526,0.25,0.80,"[[6000], [5000]]"


#### Conclusões
1. O ponto (0.5, 0.5) é próximo do mínimo local e por consequência converge para o mínimo local
2. O ponto (-1, -1) é próximo de (0, 0) e por consequência converge para valores próximos de (0, 0)
3. O ponto (5, -0.5), que pertencem a segunda parte da função, para passos grandes pula e converge pra próximo de (0, 0) e para passos pequenos converge para o mínimo local
4. O ponto (-5, 0.5), que pertencem a primeira parte da função, para passos grandes pula e converge para o mínimo local e para passos pequenos converge para próximo de (0, 0)

### Método de Newton

In [131]:
def met_newton(x,tol,maxiter,gama,n, debug=False):
    x = np.array(x)
    m = 0
    k = 0
    df = 1
    inv = 1
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        d = - (np.linalg.inv(Heissiana(x)))@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        k = k + ar[1]
        t = ar[0]
        x = x + t*d
        m = m + 1
        k = k+1
        if debug:
            df *= definida_positiva_2x2(Heissiana(x))
            inv *=  is_invertible(Heissiana(x))
            print("\nIteração: " , m)
            print(20*"-")
            print("Ponto:\n" ,x)
            print("Hessiana:\n", Heissiana(x))
            print("Definida Positiva:\n", definida_positiva_2x2(Heissiana(x)))
            print("Singular:\n", not is_invertible(Heissiana(x)))
    if debug:
        return [x,m,k, df, inv] 
    else: 
        return [x,m,k] 

In [132]:
tol = 0.0001
maxiter = 1000
result = test_combinations(search, met_newton, maxiter, tol)       
newton_res = pd.DataFrame.from_records(result)   
newton_res[newton_res["eta"] == 0.25]

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


,Armijo Iterations,Method Iterations,Optical Point Found,Optical Value Found,Time,eta,gamma,start
0,6,6,"[[0.6763327501437573], [0.6763281514789622]]",3.167673e+00,0.003649,0.25,0.80,"[[0.5], [0.5]]"
2,6,6,"[[0.6763327501437573], [0.6763281514789622]]",3.167673e+00,0.001025,0.25,0.65,"[[0.5], [0.5]]"
4,6,6,"[[0.6763327501437573], [0.6763281514789622]]",3.167673e+00,0.001211,0.25,0.50,"[[0.5], [0.5]]"
6,154962,1000,"[[-0.06787345025363754], [-0.14197059824577488]]",3.008728e-02,4.694794,0.25,0.80,"[[-1], [-1]]"
8,90775,1000,"[[-0.3797911180521609], [-0.2960744738152126]]",4.287167e-01,3.096317,0.25,0.65,"[[-1], [-1]]"
10,26,14,"[[-2.5530470568192873e-05], [-2.45032780931528...",1.252216e-09,0.003414,0.25,0.50,"[[-1], [-1]]"
12,162137,1000,"[[-0.3594177907609244], [-0.309642320351908]]",4.186475e-01,7.101004,0.25,0.80,"[[-6000], [-5000]]"
14,90108,1000,"[[-0.4237923470116344], [0.26260390832536934]]",2.496572e-01,2.855705,0.25,0.65,"[[-6000], [-5000]]"
16,29,16,"[[-1.5747819714917455e-14], [-8.68402572073989...",3.234061e-28,0.004539,0.25,0.50,"[[-6000], [-5000]]"
18,18,12,"[[0.6763342609151929], [0.6763225743104986]]",3.167673e+00,0.004767,0.25,0.80,"[[6000], [5000]]"


#### Análise dos resultados
 Newton não parece se comportar tão bem como o grandiente quando se trata da descontinuidade. Hipotese: a Hessiana nem sempre fica definida positiva próximo das descontinuidades, ou seja, as direções escolhidas não são de descida.

##### 1. Ponto (-5, -0.5) com gamma 0.8

In [133]:
newt_1 = met_newton([[-5], [-0.5]], 0.0001, 1000, 0.8, 0.25, True)
newt_1[3], newt_1[4]


Iteração:  1
--------------------
Ponto:
 [[-0.99482541]
 [-0.09221935]]
Hessiana:
 [[2.88737271 0.88737271]
 [0.88737271 1.69200757]]
Definida Positiva:
 True
Singular:
 False

Iteração:  2
--------------------
Ponto:
 [[-0.14682474]
 [-0.22794649]]
Hessiana:
 [[11.20124722  9.20124722]
 [ 9.20124722  3.03655357]]
Definida Positiva:
 False
Singular:
 False

Iteração:  3
--------------------
Ponto:
 [[-0.03084733]
 [-0.29189101]]
Hessiana:
 [[12.42932635 10.42932635]
 [10.42932635  3.52516634]]
Definida Positiva:
 False
Singular:
 False

Iteração:  4
--------------------
Ponto:
 [[-0.03084733]
 [-0.29189101]]
Hessiana:
 [[12.42932635 10.42932635]
 [10.42932635  3.52516634]]
Definida Positiva:
 False
Singular:
 False

Iteração:  5
--------------------
Ponto:
 [[-0.03084733]
 [-0.29189101]]
Hessiana:
 [[12.42932635 10.42932635]
 [10.42932635  3.52516634]]
Definida Positiva:
 False
Singular:
 False

Iteração:  6
--------------------
Ponto:
 [[-0.03084733]
 [-0.29189101]]
Hessiana:
 [[12.

(0, 1)

- Como a Hessiana não é definida positiva nas iterações finais de (-5, -0.5) a direção  escolhida nas últimas iterações não é descida e não há convergência

##### 2. Ponto (-1, -1) com gamma 0.5

In [134]:
newt_2 = met_newton([[-1], [-1]], 0.0001, 1000, 0.5, 0.25, True)
newt_2[3], newt_2[4]


Iteração:  1
--------------------
Ponto:
 [[-0.52291624]
 [-0.45851883]]
Hessiana:
 [[3.19789532 1.19789532]
 [1.19789532 1.66588578]]
Definida Positiva:
 True
Singular:
 False

Iteração:  2
--------------------
Ponto:
 [[-0.31276487]
 [ 0.16437405]]
Hessiana:
 [[6.2764348  4.2764348 ]
 [4.2764348  3.10661683]]
Definida Positiva:
 True
Singular:
 False

Iteração:  3
--------------------
Ponto:
 [[-0.1416903 ]
 [-0.07712231]]
Hessiana:
 [[11.79987762  9.79987762]
 [ 9.79987762  4.09048178]]
Definida Positiva:
 False
Singular:
 False

Iteração:  4
--------------------
Ponto:
 [[-0.11621509]
 [-0.06342811]]
Hessiana:
 [[9.21822256 7.21822256]
 [7.21822256 3.7222609 ]]
Definida Positiva:
 False
Singular:
 False

Iteração:  5
--------------------
Ponto:
 [[-0.09860269]
 [-0.04330514]]
Hessiana:
 [[5.64581867 3.64581867]
 [3.64581867 2.95868747]]
Definida Positiva:
 True
Singular:
 False

Iteração:  6
--------------------
Ponto:
 [[-0.06176116]
 [-0.07006205]]
Hessiana:
 [[4.70795647 2.7079

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


(0, 1)

- A Hessiana não é definida positiva em todas as iterações, mas Newton converge

##### 3. Ponto (6000, 5000) com gamma 0.8

In [135]:
newt_3 = met_newton([[6000], [5000]], 0.0001, 1000, 0.8, 0.35, True)
newt_3[3], newt_3[4]


Iteração:  1
--------------------
Ponto:
 [[1200.00000769]
 [1000.02481245]]
Hessiana:
 [[2.00000000e+00 1.45384545e-13]
 [1.45384545e-13 2.00000000e+00]]
Definida Positiva:
 True
Singular:
 False

Iteração:  2
--------------------
Ponto:
 [[240.00000163]
 [200.00496266]]
Hessiana:
 [[2.00000000e+00 9.10470209e-11]
 [9.10470209e-11 2.00000003e+00]]
Definida Positiva:
 True
Singular:
 False

Iteração:  3
--------------------
Ponto:
 [[48.00000262]
 [40.00099689]]
Hessiana:
 [[2.00000006e+00 5.74755377e-08]
 [5.74755377e-08 2.00000329e+00]]
Definida Positiva:
 True
Singular:
 False

Iteração:  4
--------------------
Ponto:
 [[9.60006  ]
 [8.0003105]]
Hessiana:
 [[2.00003776e+00 3.77600919e-05]
 [3.77600919e-05 2.00044235e+00]]
Definida Positiva:
 True
Singular:
 False

Iteração:  5
--------------------
Ponto:
 [[1.92178772]
 [1.60313193]]
Hessiana:
 [[2.03011421 0.03011421]
 [0.03011421 2.07919163]]
Definida Positiva:
 True
Singular:
 False

Iteração:  6
--------------------
Ponto:
 [[0

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


(1, 1)

- Newton converge e a Hessiana é definida positiva em todas as iterações, ou seja, todas as iterações são de descida

## Método Quase Newton


#### DFP

In [136]:
def DFP(x_atual,x_antigo,H):
    x_atual = np.array(x_atual)
    x_antigo = np.array(x_antigo)
    p = x_atual - x_antigo
    q = Gradiente(x_atual)- Gradiente(x_antigo)
    if np.linalg.norm(p) != 0 and np.linalg.norm(q) != 0:
        return H + (p@np.transpose(p))/(np.transpose(p)@q) - (H@q@np.transpose(q)@H)/(np.transpose(q)@H@q)
    else: 
        print('pq <= 0')
        return H

In [137]:
def met_quase_newton_DFP(x,tol,maxiter,gama,n):
    x = np.array(x)
    m = 0
    k =0
    H = np.identity(2)
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        x_antigo = x
        d = -1*H@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        t = ar[0]
        x = x + t*d
        H = DFP(x,x_antigo,H)
        m = m + 1
        k = k + ar[1]
        
    return [x,m,k] 

In [138]:
tol = 0.0001
maxiter = 1000
result = test_combinations(search, met_quase_newton_DFP, maxiter, tol)       
dfp_res = pd.DataFrame.from_records(result)    
dfp_res[dfp_res['eta'] == 0.25]

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


,Armijo Iterations,Method Iterations,Optical Point Found,Optical Value Found,Time,eta,gamma,start
0,10,5,"[[0.6763353628281227], [0.6763353628281227]]",3.167673e+00,0.001517,0.25,0.80,"[[0.5], [0.5]]"
2,6,4,"[[0.676332691443663], [0.676332691443663]]",3.167673e+00,0.001434,0.25,0.65,"[[0.5], [0.5]]"
4,4,5,"[[0.6763326473319318], [0.6763326473319318]]",3.167673e+00,0.001289,0.25,0.50,"[[0.5], [0.5]]"
6,10,6,"[[-3.516154439006929e-05], [-3.516154439009241...",2.472668e-09,0.004365,0.25,0.80,"[[-1], [-1]]"
8,3,4,"[[-6.071532165918825e-18], [-1.040834085586084...",1.451971e-34,0.000991,0.25,0.65,"[[-1], [-1]]"
10,3,5,"[[5.2888737226558513e-17], [-5.532141585107286...",5.857678e-33,0.001166,0.25,0.50,"[[-1], [-1]]"
12,11,13,"[[0.676335010060589], [0.6763313704115276]]",3.167673e+00,0.007407,0.25,0.80,"[[-6000], [-5000]]"
14,10,13,"[[0.6763277833164886], [0.6763333982829317]]",3.167673e+00,0.002935,0.25,0.65,"[[-6000], [-5000]]"
16,18,20,"[[-3.7239897350224685e-05], [1.786778325142697...",1.706068e-09,0.007035,0.25,0.50,"[[-6000], [-5000]]"
18,11,11,"[[-1.2772953068963789e-05], [8.839925742881646...",2.412926e-10,0.003186,0.25,0.80,"[[6000], [5000]]"


- Os resultads deste método ficaram semelhantes ao método de newton

#### BFGS

In [139]:
def Armijo(x,gama,d,n):
    t = 1
    k = 0
    d = np.array(d)
    x = np.array(x)
    while f(x + t*d) > f(x) + n*t*(np.transpose(Gradiente(x))@d):
        t=gama*t
        k = k+1
    return [t,k]

In [140]:
def BFGS(x_atual,x_antigo,H, debug):
    x_atual = np.array(x_atual)
    x_antigo = np.array(x_antigo)

    p = x_atual - x_antigo
    q = Gradiente(x_atual)- Gradiente(x_antigo)
    if debug:
        print("Vetor p: ",p)
        print("Vetor q: ",p)
    return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)

In [141]:
def met_quase_newton_BFGS(x,tol,maxiter,gama,n, debug=False):
    x = np.array(x)
    m = 0
    k =0
    df = 1
    H = np.identity(2)
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        x_antigo = x
        d = -1*H@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        t = ar[0]
        x = x + t*d
        if debug:
            print("\nIteração: " , m)
            print(20*"-")
            print("Ponto:\n" ,x)
            print("Hessiana:\n", H)
            print("Definida Positiva:\n", definida_positiva_2x2(H))
        H = BFGS(x,x_antigo,H, debug)
        m = m + 1
        k = k + ar[1]
    if debug:
        return [x,m,k,df] 
    else:
        return [x,m,k]  


##### Análise do Erro do método
No primeiro teste que fizemos a implementação dá erro, pois p ou q são iguais a zero.

Teorema: Quando a hessiana é definida positiva e o t é calculado por busca exata, p*q>0 e a próxima hessiana cálculada é definida positiva.

O problema ta na busca armijo ou na hessiana?

In [142]:
met_quase_newton_BFGS([[0.5], [0.5]], 0.0001, 1000, 0.8, 0.25)

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\551132956.py:10: RuntimeWarning: invalid value encountered in true_divide
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)


[array([[nan],
        [nan]]),
 5,
 193]

- Conclusão as Hessianas são definidas positivas o problema tá no uso da busca de armijo

In [143]:
tol = 0.0001
maxiter = 1000
result = test_combinations(search, met_quase_newton_BFGS, maxiter, tol)     
bfgs_res = pd.DataFrame.from_records(result)      
bfgs_res

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\551132956.py:10: RuntimeWarning: invalid value encountered in true_divide
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\551132956.py:10: RuntimeWarning: divide by zero encountered in true_divide
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\551132956.py:10: RuntimeWarning: invalid value encountered in matmul
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np

,Armijo Iterations,Method Iterations,Optical Point Found,Optical Value Found,Time,eta,gamma,start
0,193,5,"[[nan], [nan]]",NaN,0.011305,0.25,0.80,"[[0.5], [0.5]]"
1,257,6,"[[nan], [nan]]",NaN,0.017937,0.10,0.80,"[[0.5], [0.5]]"
2,175,8,"[[0.6763260400321802], [0.6763260400321802]]",3.167673,0.014985,0.25,0.65,"[[0.5], [0.5]]"
3,107,5,"[[nan], [nan]]",NaN,0.010139,0.10,0.65,"[[0.5], [0.5]]"
4,169,12,"[[0.676330667239791], [0.676330667239791]]",3.167673,0.010320,0.25,0.50,"[[0.5], [0.5]]"
5,77,6,"[[nan], [nan]]",NaN,0.005321,0.10,0.50,"[[0.5], [0.5]]"
6,194,4,"[[nan], [nan]]",NaN,0.008443,0.25,0.80,"[[-1], [-1]]"
7,194,4,"[[nan], [nan]]",NaN,0.009939,0.10,0.80,"[[-1], [-1]]"
8,102,4,"[[nan], [nan]]",NaN,0.004441,0.25,0.65,"[[-1], [-1]]"
9,102,4,"[[nan], [nan]]",NaN,0.004361,0.10,0.65,"[[-1], [-1]]"


- Para tentarmos contornar o problema pesquisamos na literatura e encontramos que com certas condições a modificação Armijo poderia retornar um t que é aproximado utilizando também a Heissiana e, assim, melhorar a escolha do passo.

    http://koreascience.kr/article/JAKO200807663775076.pdf

In [144]:
def Armijo_mod(x,d,H,gama):
    t = 1
    alfa = 0.2
    k = 0
    g = Gradiente(x)
    cache =  alfa*(np.transpose(Gradiente(x))@d + 0.5*t*np.transpose(d)@H@d)
    while f(x + t*d) > f(x) +t*cache:
        t=gama*t
        k = k+1
    return [t,k]

In [145]:
def met_quase_newton_BFGS_mod(x,tol,maxiter,gama,n, debug=False):
    x = np.array(x)
    m = 0
    k =0
    df = 1
    H = np.identity(2)
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        x_antigo = x
        d = -1*H@Gradiente(x)
        ar = Armijo_mod(x,d,H,gama)
        t = ar[0]
        x = x + t*d
        if debug:
            print("\nIteração: " , m)
            print(20*"-")
            print("Ponto:\n" ,x)
            print("Hessiana:\n", H)
            print("Definida Positiva:\n", definida_positiva_2x2(H))
        H = BFGS(x,x_antigo,H, debug)
        m = m + 1
        k = k + ar[1]
    if debug:
        return [x,m,k,df] 
    else:
        return [x,m,k] 

In [146]:
met_quase_newton_BFGS_mod([[0.5], [0.5]], 0.0001, 1000, 0.5, 0.25,True)


Iteração:  0
--------------------
Ponto:
 [[0.78697961]
 [0.78697961]]
Hessiana:
 [[1. 0.]
 [0. 1.]]
Definida Positiva:
 True
Vetor p:  [[0.28697961]
 [0.28697961]]
Vetor q:  [[0.28697961]
 [0.28697961]]

Iteração:  1
--------------------
Ponto:
 [[-0.870969]
 [-0.870969]]
Hessiana:
 [[1.71421443 0.71421443]
 [0.71421443 1.71421443]]
Definida Positiva:
 True
Vetor p:  [[-1.65794861]
 [-1.65794861]]
Vetor q:  [[-1.65794861]
 [-1.65794861]]

Iteração:  2
--------------------
Ponto:
 [[76.7562797]
 [76.7562797]]
Hessiana:
 [[20.56249781 19.56249781]
 [19.56249781 20.56249781]]
Definida Positiva:
 True
Vetor p:  [[77.6272487]
 [77.6272487]]
Vetor q:  [[77.6272487]
 [77.6272487]]

Iteração:  3
--------------------
Ponto:
 [[-2.9730805e+08]
 [-2.9730805e+08]]
Hessiana:
 [[968351.94423314 968350.94423314]
 [968350.94423314 968351.94423314]]
Definida Positiva:
 True
Vetor p:  [[-2.97308126e+08]
 [-2.97308126e+08]]
Vetor q:  [[-2.97308126e+08]
 [-2.97308126e+08]]

Iteração:  4
----------------

C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\4264350725.py:6: RuntimeWarning: overflow encountered in matmul
  cache =  alfa*(np.transpose(Gradiente(x))@d + 0.5*t*np.transpose(d)@H@d)
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\551132956.py:10: RuntimeWarning: overflow encountered in matmul
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)
C:\Users\mwuillau\AppData\Local\Temp\ipykernel_2572\551132956.py:10: RuntimeWarning: invalid value encountered in subtract
  return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)


[array([[nan],
        [nan]]),
 7,
 3,
 1]

- Como as condições para esses algoritmos modificado não são atendidos (não atende a cotinuidade Lipschitz), não foi possível obter um resultado.

##### Análise dos resultados
- Somente para o ponto próximo do mínimo local com passo pequeno o método converge

# Comparação dos Métodos
<hr/>

#### Comparação entre DFP e Gradiente

1. Número de Iterações

In [147]:
dfp_conv = dfp_res['Method Iterations'] != 1000
(
    grad_res.loc[dfp_conv, 'Method Iterations'] >= dfp_res.loc[dfp_conv, 'Method Iterations']
).sum(), (dfp_res['Method Iterations'] != 1000).sum()

(29, 36)

- O método DFP apresenta menos iterações

2. Tempo

In [148]:
dfp_conv = dfp_res['Method Iterations'] != 1000
(
    grad_res.loc[dfp_conv, 'Time'] >= dfp_res.loc[dfp_conv, 'Time']
).sum(), (dfp_res['Method Iterations'] != 1000).sum()

(16, 36)

In [149]:
print('DFP vs Grad')
dfp_res.loc[18:23, ['Time', 'start']], grad_res.loc[18:23, ['Time', 'start']]

DFP vs Grad


(        Time             start
 18  0.003186  [[6000], [5000]]
 19  0.007210  [[6000], [5000]]
 20  0.005312  [[6000], [5000]]
 21  0.004704  [[6000], [5000]]
 22  0.006980  [[6000], [5000]]
 23  0.004244  [[6000], [5000]],
         Time             start
 18  0.002526  [[6000], [5000]]
 19  0.005140  [[6000], [5000]]
 20  0.002334  [[6000], [5000]]
 21  0.001411  [[6000], [5000]]
 22  0.002452  [[6000], [5000]]
 23  0.004861  [[6000], [5000]])

- mesmo analisando apenas os números pontos iniciais grandes, o gradiente ainda consegue ser mais lento que o DFP em alguns testes

3. Número de iterações de armijo

In [159]:
dfp_conv = dfp_res['Armijo Iterations'] != 1000
(
    grad_res.loc[dfp_conv, 'Armijo Iterations'] >= dfp_res.loc[dfp_conv, 'Method Iterations']
).sum(), (dfp_res['Method Iterations'] != 1000).sum()

(36, 36)

- O gradiente apresenta mais iterações de Armijo em todos os testes, 

#### Comparação entre Newton e Gradiente

Acreditamos que Newton performou melhor que o gradiente devido aos critérios abaixo, isso pode ter relação com o fato de Newton realizar uma aproximação quadrática da função e a nossa função ter aspectos de uma função quadrática, delocada por $3^{\frac{1}{x_1+x_2}}$.

1. Número de Iterações

In [151]:
newton_conv = newton_res['Method Iterations'] != 1000
(
    grad_res.loc[newton_conv, 'Method Iterations'] >= newton_res.loc[newton_conv, 'Method Iterations']
).sum(), (newton_res['Method Iterations'] != 1000).sum()

(22, 24)

- O método de newton apresenta menos iterações

2. Tempo

In [152]:
newton_conv = newton_res['Method Iterations'] != 1000
(
    grad_res.loc[newton_conv, 'Time'] >= newton_res.loc[newton_conv, 'Time']
).sum(), (newton_res['Method Iterations'] != 1000).sum()

(10, 24)

In [153]:
print('Newton vs Grad')
newton_res.loc[18:23, ['Time', 'start']], grad_res.loc[18:23, ['Time', 'start']]

Newton vs Grad


(        Time             start
 18  0.004767  [[6000], [5000]]
 19  0.002984  [[6000], [5000]]
 20  0.003758  [[6000], [5000]]
 21  0.003365  [[6000], [5000]]
 22  0.005715  [[6000], [5000]]
 23  0.004902  [[6000], [5000]],
         Time             start
 18  0.002526  [[6000], [5000]]
 19  0.005140  [[6000], [5000]]
 20  0.002334  [[6000], [5000]]
 21  0.001411  [[6000], [5000]]
 22  0.002452  [[6000], [5000]]
 23  0.004861  [[6000], [5000]])

- mesmo analisando apenas os números pontos iniciais grandes, o gradiente ainda consegue ser mais lento que o newton em alguns casos analisados

3. Número de iterações de armijo

In [160]:
newton_conv = newton_res['Armijo Iterations'] != 1000
(
    grad_res.loc[newton_conv, 'Armijo Iterations'] >= newton_res.loc[newton_conv, 'Method Iterations']
).sum(), (newton_res['Method Iterations'] != 1000).sum()

(24, 24)

- O gradiente apresenta mais iterações de Armijo em todos os testes, 

#### Comparação entre DFP e Newton

1. Número de Iterações

In [161]:
newton_conv = newton_res['Method Iterations'] != 1000
dfp_conv = dfp_res['Method Iterations'] != 1000
(
    newton_res.loc[newton_conv&dfp_conv, 'Method Iterations'] >= dfp_res.loc[newton_conv&dfp_conv, 'Method Iterations']
).sum(), (newton_conv&dfp_conv).sum()

(15, 24)

- O método DFP apresenta menos iterações, na maioria das iterações

2. Tempo

In [162]:
newton_conv = newton_res['Method Iterations'] != 1000
dfp_conv = dfp_res['Method Iterations'] != 1000
(
    newton_res.loc[newton_conv&dfp_conv, 'Time'] >= dfp_res.loc[newton_conv&dfp_conv, 'Time']
).sum(), (newton_conv&dfp_conv).sum()

(8, 24)

In [157]:
print('DFP vs Newton')
newton_res.loc[18:23, ['Time', 'start']], grad_res.loc[18:23, ['Time', 'start']]

DFP vs Newton


(        Time             start
 18  0.004767  [[6000], [5000]]
 19  0.002984  [[6000], [5000]]
 20  0.003758  [[6000], [5000]]
 21  0.003365  [[6000], [5000]]
 22  0.005715  [[6000], [5000]]
 23  0.004902  [[6000], [5000]],
         Time             start
 18  0.002526  [[6000], [5000]]
 19  0.005140  [[6000], [5000]]
 20  0.002334  [[6000], [5000]]
 21  0.001411  [[6000], [5000]]
 22  0.002452  [[6000], [5000]]
 23  0.004861  [[6000], [5000]])

3. Número de iterações de armijo

In [163]:
newton_conv = newton_res['Method Iterations'] != 1000
dfp_conv = dfp_res['Method Iterations'] != 1000
(
    newton_res.loc[newton_conv&dfp_conv, 'Armijo Iterations'] >= dfp_res.loc[newton_conv&dfp_conv, 'Armijo Iterations']
).sum(), (newton_conv&dfp_conv).sum()

(19, 24)

- O newton apresenta mais iterações de Armijo na maioria dos casos, quando comparado com DFP 